In [1]:
import os
import sys
import SimpleITK as sitk
import numpy as np
import cv2
from skimage import morphology
from matplotlib import pyplot as plot
import pandas as pd

In [19]:
# 修改文件名称
if False:
    model_name = "rvm"
    result_dir = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}"
    file_list = os.listdir(result_dir)
    try:
        file_list.remove(".DS_Store")
    except ValueError:
        pass
    try:
        file_list.remove("._.DS_Store")
    except ValueError:
        pass

    save_path = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/results/{model_name}"

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for path in file_list:
        data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(result_dir, path)))
        name = path[:-18]
        out = sitk.GetImageFromArray(data)
        sitk.WriteImage(out, f"{save_path}/{name}.nii.gz")

In [2]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "rvm"
result_dir = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}"
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values
)

pn_all = {}
pne_all = {}
mask_all = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name))
        )
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(
        spacing_pixels[np.argwhere(spacing_pixels[:, 0] == file_name[:-7])][0][0][1]
    )

    pn = []
    pne = []
    mask = {}
    mask_out_list = []
    mask_in_list = []
    mask_both_list = []
    mask_both_erode_list = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(
            both_mask, kernel=(3, 3), iterations=spacing // 10
        )  # 外膜腐蚀mask

        mask_out_list.append(out_mask)
        mask_in_list.append(in_mask)
        mask_both_list.append(both_mask)
        mask_both_erode_list.append(both_mask_erode)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f"pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}"
            )

    mask["out"] = mask_out_list
    mask["in"] = mask_in_list
    mask["both"] = mask_both_list
    mask["both_erode"] = mask_both_erode_list

    mask_all[file_name] = mask
    pn_all[file_name] = pd.Series(pn)
    pne_all[file_name] = pd.Series(pne)


if False:
    with pd.ExcelWriter(f"{model_name}.xlsx") as writer:
        df_pn_all = pd.DataFrame(data=pn_all)
        df_pn_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

    with pd.ExcelWriter(f"{model_name}_erode.xlsx") as writer:
        df_pne_all = pd.DataFrame(data=pne_all)
        df_pne_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

In [3]:
def remove_isolate(inputs, threshold_area=0.5):
    mask = np.zeros((inputs.shape[0], inputs.shape[1]), dtype=np.uint8)
    mask[np.sum(inputs, axis=-1) > 0] = 1
    mask = morphology.remove_small_objects(
        mask.astype(np.bool8), np.sum(mask) * threshold_area, connectivity=8
    ).astype(np.uint8)
    # outputs = np.expand_dims(mask, -1) * inputs
    outputs = mask * inputs

    return outputs

In [4]:
def minimum_external_circle(img):
    contours, _ = cv2.findContours(
        img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE
    )  # 提取轮廓
    img = np.expand_dims(img, -1).repeat(3, -1)

    cnt = contours[0]
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))  # 最小内接圆圆心
    radius = int(radius)  # 最小内接圆直径
    cv2.circle(img, center, radius, (0, 255, 0), 2)
    cv2.circle(img, center, 1, (0, 255, 0), 2)
    return (x, y), radius, img

In [5]:
# 把笛卡尔坐标转化为极坐标
def to_polar(vector, x, y):
    v_length = np.sqrt((vector[1] - int(x)) ** 2 + (vector[0] - int(y)) ** 2)
    v_angle = np.arctan2(vector[0] - int(y), vector[1] - int(x))
    return (v_length, np.around(v_angle, 2))

In [6]:
def circle_max_distance(polar):
    angle_range = np.linspace(-3.14, 3.14, int((3.14 * 2) / 0.01 + 1))
    polar = np.array(polar)
    polar_angle = polar[:, 1]
    radial = []
    for angle in angle_range:
        idx_list = np.argwhere(polar_angle == np.around(angle, 2))
        if len(idx_list) != 0:
            distance = np.max(polar[idx_list][:, 0, 0]) - np.min(
                polar[idx_list][:, 0, 0]
            )
            radial.append(int(distance))

    return max(radial)

In [11]:
data_distance = {}
for i, data_tuple in enumerate(list(mask_all.items())):
    data = data_tuple[1]["out"]
    name = data_tuple[0]
    frame_distance = []
    # if name[:-7] == "IM_0044":
    for j, frame in enumerate(data):
        frame_pad = np.pad(
            frame, ((100, 100), (100, 100)), "constant", constant_values=0
        )
        frame_pad = cv2.morphologyEx(
            frame_pad, cv2.MORPH_CLOSE, kernel=np.ones((15, 15), np.uint8)
        )
        frame = frame_pad[100:-100, 100:-100]
        frame = remove_isolate(frame)

        area_pad = np.pad(
            data_tuple[1]["both"][j],
            ((100, 100), (100, 100)),
            "constant",
            constant_values=0,
        )
        area_pad = cv2.morphologyEx(
            area_pad, cv2.MORPH_CLOSE, kernel=np.ones((15, 15), np.uint8)
        )
        area = area_pad[100:-100, 100:-100]
        area = remove_isolate(area)

        (x, y), radius, visual_frame = minimum_external_circle(
            np.array(area, dtype=np.uint8)
        )

        frame_index = np.argwhere(frame == 1)

        polar = []
        for idx in range(len(frame_index)):
            polar.append(to_polar(frame_index[idx], x, y))
        polar = sorted(polar, key=lambda item: item[1])

        frame_distance.append(circle_max_distance(polar))

        # plot.figure(j)
        # plot.imshow(visual_frame)
        # plot.title(str(circle_max_distance(polar)))

    data_distance[name] = frame_distance

In [14]:
data_distance.keys()

dict_keys(['202203141429300009VAS.nii.gz', '202204020917510011VAS.nii.gz', 'IM_0061.nii.gz', 'IM_0049.nii.gz', '202203081456170036VAS.nii.gz', 'IM_0039.nii.gz', '202204110951560016VAS.nii.gz', '202204070858070033VAS.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', 'IM_0010.nii.gz', '_20210128075939_0818310.nii.gz', '202203251425020010VAS.nii.gz', '202203241107360023VAS.nii.gz', 'IM_0033.nii.gz', '202203211455210024VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '202203241105420022VAS.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '02220820210305_XJ_20210305082931543.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0084.nii.gz', '_20210224093839_0943280.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', 'IM_0044.nii.gz', 'IM_0079.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '202203140858390011VAS.nii.gz'])

In [15]:
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
data_distance_max = {
    k[:-7]: (
        max(v)
        / int(spacing_pixels[np.argwhere(spacing_pixels[:, 0] == k[:-7])][0][0][1])
    )
    * 5
    for k, v in data_distance.items()
}
data_distance_max = sorted(data_distance_max.items(), key=lambda x: x[1])
data_distance_max

[('IM_0039', 0.847457627118644),
 ('IM_0049', 0.9322033898305084),
 ('202204020917510011VAS', 0.958904109589041),
 ('IM_0084', 0.9859154929577464),
 ('24060820210210_SMY_20210210081527458', 1.056338028169014),
 ('202203251425020010VAS', 1.0666666666666667),
 ('29081020210319_YZX_20210319101543139', 1.1643835616438356),
 ('IM_0010', 1.1764705882352942),
 ('IM_0061', 1.1864406779661016),
 ('202203140858390011VAS', 1.2195121951219512),
 ('IM_0033', 1.25),
 ('43291020210205_CQH_20210205103732049', 1.3888888888888888),
 ('IM_0044', 1.4166666666666665),
 ('_20210128075939_0818310', 1.5760869565217392),
 ('202204070858070033VAS', 1.6666666666666665),
 ('202203211455210024VAS', 1.689189189189189),
 ('202204110951560016VAS', 1.7999999999999998),
 ('02220820210305_XJ_20210305082931543', 1.8181818181818183),
 ('202203081456170036VAS', 1.8493150684931505),
 ('202203141429300009VAS', 2.236842105263158),
 ('_20210224093839_0943280', 2.5),
 ('39561320210331_DJF_20210331140409947', 2.708333333333333),

In [37]:
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
data_distance_max_index = {k[:-7]: v.index(max(v)) for k, v in data_distance.items()}
# data_distance_max_index = sorted(data_distance_max_index.items(),
#                                  key=lambda x: x[1])
data_distance_max_index

{'202203141429300009VAS': 8,
 '202204020917510011VAS': 0,
 'IM_0061': 37,
 'IM_0049': 150,
 '202203081456170036VAS': 68,
 'IM_0039': 3,
 '202204110951560016VAS': 152,
 '202204070858070033VAS': 250,
 '06570820220302_CA MASHAOYONG_20220302091057736': 0,
 'IM_0010': 180,
 '_20210128075939_0818310': 8,
 '202203251425020010VAS': 4,
 '202203241107360023VAS': 93,
 'IM_0033': 49,
 '202203211455210024VAS': 110,
 '24060820210210_SMY_20210210081527458': 53,
 '202203241105420022VAS': 173,
 '29081020210319_YZX_20210319101543139': 223,
 '02220820210305_XJ_20210305082931543': 0,
 '43291020210205_CQH_20210205103732049': 0,
 'IM_0084': 24,
 '_20210224093839_0943280': 44,
 '06570820220302_CA MASHAOYONG_20220302091109144': 0,
 'IM_0044': 136,
 'IM_0079': 0,
 '39561320210331_DJF_20210331140409947': 115,
 '202203140858390011VAS': 12}

In [34]:
# 视频中外膜径向最大值(无斑块33,39,44,49,79)
data_distance_length = {k[:-7]: len(v) for k, v in data_distance.items()}
data_distance_length

{'202203141429300009VAS': 45,
 '202204020917510011VAS': 140,
 'IM_0061': 143,
 'IM_0049': 151,
 '202203081456170036VAS': 70,
 'IM_0039': 151,
 '202204110951560016VAS': 198,
 '202204070858070033VAS': 254,
 '06570820220302_CA MASHAOYONG_20220302091057736': 116,
 'IM_0010': 196,
 '_20210128075939_0818310': 77,
 '202203251425020010VAS': 76,
 '202203241107360023VAS': 98,
 'IM_0033': 151,
 '202203211455210024VAS': 113,
 '24060820210210_SMY_20210210081527458': 161,
 '202203241105420022VAS': 174,
 '29081020210319_YZX_20210319101543139': 286,
 '02220820210305_XJ_20210305082931543': 78,
 '43291020210205_CQH_20210205103732049': 62,
 'IM_0084': 151,
 '_20210224093839_0943280': 47,
 '06570820220302_CA MASHAOYONG_20220302091109144': 86,
 'IM_0044': 137,
 'IM_0079': 147,
 '39561320210331_DJF_20210331140409947': 193,
 '202203140858390011VAS': 41}

In [54]:
import os
import shutil

path = r"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/dataset/0801_0815/dataset_0810/image"
save_path = "/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄帧_AI选图/selcet/"

# 通过DFS实现统一逻辑处理同一层次的文件对象
for root, dirs, files in os.walk(path):
    if not dirs:
        keywords = list(data_distance_max_index.keys())
        dir_name = root.split("/")[-1]
        if dir_name in keywords:
            if not os.path.exists(save_path + dir_name):
                os.makedirs(save_path + dir_name)
            ext = ".png"
            start_index = int(
                sorted([data for data in files if "._" not in data])[0].split(".")[0]
            )
            for file_name in files:
                if (
                    str(data_distance_max_index[dir_name] + start_index).zfill(4) + ext
                    == file_name
                ):
                    image = cv2.imread(os.path.join(root, file_name))
                    cv2.imwrite(
                        os.path.join(
                            save_path,
                            f"{dir_name}_{data_distance_max_index[dir_name]}.png",
                        ),
                        image,
                    )

print("运行结束")

运行结束


In [20]:
data_distance["202204020917510011VAS.nii.gz"]

[14,
 13,
 13,
 12,
 10,
 11,
 12,
 11,
 12,
 12,
 13,
 12,
 11,
 11,
 8,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 10,
 9,
 9,
 9,
 9,
 8,
 8,
 9,
 10,
 10,
 10,
 10,
 11,
 10,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 10,
 11,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 10,
 10,
 9,
 9,
 10,
 9,
 9,
 8,
 8,
 9,
 9,
 8,
 9,
 8,
 9,
 9,
 10,
 9,
 10,
 9,
 9,
 10,
 10,
 10,
 11,
 11,
 10,
 9,
 11,
 11,
 11,
 10,
 8,
 10,
 11,
 11,
 9,
 9,
 9,
 10,
 10,
 11,
 11,
 10,
 10,
 11,
 9,
 9,
 9,
 9,
 10,
 8,
 9,
 9,
 9,
 10,
 9,
 9,
 8,
 8]

In [52]:
os.path.join(save_path, f"{dir_name}_{data_distance_max_index[dir_name]}")

'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄帧_AI选图/selcet/202204110951560016VAS_152'

In [42]:
# 最大值(无斑块33,39,44,49,79)
pne_max = {}
for idx in pne_all.keys():
    if idx == "IM_0039.nii.gz":
        pne_max[idx] = max(list(pne_all[idx])[:100])
    elif idx == "IM_0033.nii.gz":
        pne_max[idx] = max(list(pne_all[idx])[:140])
    else:
        pne_max[idx] = max(list(pne_all[idx]))
pne_max = sorted(pne_max.items(), key=lambda x: x[1])
pne_max

[('IM_0049.nii.gz', 0.21183973381394705),
 ('IM_0044.nii.gz', 0.22655538694992416),
 ('IM_0084.nii.gz', 0.22656485481764288),
 ('IM_0079.nii.gz', 0.2588084561178732),
 ('IM_0061.nii.gz', 0.26379542395693134),
 ('02220820210305_XJ_20210305082931543.nii.gz', 0.26424151292579956),
 ('24060820210210_SMY_20210210081527458.nii.gz', 0.29443928044179557),
 ('29081020210319_YZX_20210319101543139.nii.gz', 0.3004717231144326),
 ('43291020210205_CQH_20210205103732049.nii.gz', 0.3109327983951855),
 ('202203140858390011VAS.nii.gz', 0.32623407643312097),
 ('IM_0039.nii.gz', 0.33350533195734433),
 ('IM_0033.nii.gz', 0.33937920718025427),
 ('IM_0010.nii.gz', 0.3660315570110152),
 ('202203141429300009VAS.nii.gz', 0.3716353766116264),
 ('_20210224093839_0943280.nii.gz', 0.3982221529104448),
 ('_20210128075939_0818310.nii.gz', 0.3996132843210165),
 ('202204020917510011VAS.nii.gz', 0.4136616362192216),
 ('202204110951560016VAS.nii.gz', 0.4174679487179487),
 ('202203251425020010VAS.nii.gz', 0.42120526241335

In [40]:
# 均值(无斑块33,39,44,49,79)
pne_mean = {}
for idx in pne_all.keys():
    if idx == "IM_0039.nii.gz":
        pne_mean[idx] = np.mean(list(pne_all[idx])[:100])
    elif idx == "IM_0033.nii.gz":
        pne_mean[idx] = np.mean(list(pne_all[idx])[:140])
    else:
        pne_mean[idx] = sum(list(pne_all[idx])) / len(list(pne_all[idx]))
pne_mean = sorted(pne_mean.items(), key=lambda x: x[1])
pne_mean

[('IM_0044.nii.gz', 0.16518610495520256),
 ('IM_0084.nii.gz', 0.18486155868984105),
 ('IM_0049.nii.gz', 0.1905875960454178),
 ('IM_0079.nii.gz', 0.19647242691477013),
 ('IM_0061.nii.gz', 0.21629943270342064),
 ('IM_0010.nii.gz', 0.22232869417916418),
 ('02220820210305_XJ_20210305082931543.nii.gz', 0.2227912313828838),
 ('29081020210319_YZX_20210319101543139.nii.gz', 0.2420430883026508),
 ('202203141429300009VAS.nii.gz', 0.24401819874601804),
 ('24060820210210_SMY_20210210081527458.nii.gz', 0.24547303739920884),
 ('IM_0033.nii.gz', 0.24766277823189156),
 ('IM_0039.nii.gz', 0.24824757862024685),
 ('202203140858390011VAS.nii.gz', 0.26841002556706023),
 ('43291020210205_CQH_20210205103732049.nii.gz', 0.27031541058218134),
 ('_20210128075939_0818310.nii.gz', 0.2833127512393653),
 ('_20210224093839_0943280.nii.gz', 0.28528261127820403),
 ('202203211455210024VAS.nii.gz', 0.28905055752515507),
 ('202204110951560016VAS.nii.gz', 0.2988434839477413),
 ('202203251425020010VAS.nii.gz', 0.2992594110

In [43]:
# 方差(无斑块33,39,44,49,79)
pne_std = {}
for idx in pne_all.keys():
    if idx == "IM_0039.nii.gz":
        pne_std[idx] = np.std(list(pne_all[idx])[:100])
    elif idx == "IM_0033.nii.gz":
        pne_std[idx] = np.std(list(pne_all[idx])[:140])
    else:
        pne_std[idx] = np.std(list(pne_all[idx]))
pne_std = sorted(pne_std.items(), key=lambda x: x[1])
pne_std

[('IM_0049.nii.gz', 0.009901453531162702),
 ('IM_0084.nii.gz', 0.016283769285645164),
 ('02220820210305_XJ_20210305082931543.nii.gz', 0.01992565453759014),
 ('IM_0079.nii.gz', 0.022477434842812373),
 ('IM_0061.nii.gz', 0.022544706726708167),
 ('202203140858390011VAS.nii.gz', 0.023019488421389598),
 ('24060820210210_SMY_20210210081527458.nii.gz', 0.02723345981203385),
 ('IM_0039.nii.gz', 0.03158895964181743),
 ('202203251425020010VAS.nii.gz', 0.031981819731150087),
 ('43291020210205_CQH_20210205103732049.nii.gz', 0.03203813428738874),
 ('IM_0033.nii.gz', 0.032330526856467395),
 ('29081020210319_YZX_20210319101543139.nii.gz', 0.032770369003166634),
 ('202204020917510011VAS.nii.gz', 0.03537363533078301),
 ('202204110951560016VAS.nii.gz', 0.043649677005445704),
 ('IM_0010.nii.gz', 0.04447050097507992),
 ('_20210128075939_0818310.nii.gz', 0.05204744187353978),
 ('IM_0044.nii.gz', 0.0520781161877553),
 ('_20210224093839_0943280.nii.gz', 0.058015675378306694),
 ('202204070858070033VAS.nii.gz'

In [3]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "3dunet"
result_dir = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}"
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values
)

pn_all_1 = {}
pne_all_1 = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name))
        )
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(
        spacing_pixels[np.argwhere(spacing_pixels[:, 0] == file_name[:-7])][0][0][1]
    )

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask, kernel=(3, 3), iterations=spacing // 10)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f"pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}"
            )

    pn_all_1[file_name] = pd.Series(pn)
    pne_all_1[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f"{model_name}.xlsx") as writer:
        df_pn_all = pd.DataFrame(data=pn_all_1)
        df_pn_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

    with pd.ExcelWriter(f"{model_name}_erode.xlsx") as writer:
        df_pne_all = pd.DataFrame(data=pne_all_1)
        df_pne_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

In [11]:
print(sorted(list(pn_all_1.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [4]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "rvm"
result_dir = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}"
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values
)

pn_all_2 = {}
pne_all_2 = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name))
        )
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(
        spacing_pixels[np.argwhere(spacing_pixels[:, 0] == file_name[:-7])][0][0][1]
    )

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask, kernel=(3, 3), iterations=spacing // 10)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f"pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}"
            )

    pn_all_2[file_name] = pd.Series(pn)
    pne_all_2[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f"{model_name}.xlsx") as writer:
        df_pn_all = pd.DataFrame(data=pn_all_2)
        df_pn_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

    with pd.ExcelWriter(f"{model_name}_erode.xlsx") as writer:
        df_pne_all = pd.DataFrame(data=pne_all_2)
        df_pne_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

In [14]:
print(sorted(list(pn_all_2.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [11]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "transbts"
result_dir = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0920/{model_name}"
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

spacing_pixels = np.array(
    pd.read_excel("/Users/WangHao/Desktop/TODO/Data/动态测试集spacing.xlsx").values
)

pn_all_3 = {}
pne_all_3 = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(result_dir, file_name))
        )
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    spacing = int(
        spacing_pixels[np.argwhere(spacing_pixels[:, 0] == file_name[:-7])][0][0][1]
    )

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask, kernel=(3, 3), iterations=1)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(
                f"pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}"
            )

    pn_all_3[file_name] = pd.Series(pn)
    pne_all_3[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f"{model_name}.xlsx") as writer:
        df_pn_all = pd.DataFrame(data=pn_all_3)
        df_pn_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

    with pd.ExcelWriter(f"{model_name}_erode.xlsx") as writer:
        df_pne_all = pd.DataFrame(data=pne_all_3)
        df_pne_all.to_excel(
            writer,
            sheet_name="diameter stenosis",
            index=True,
            header=True,
            startrow=0,
            startcol=0,
        )

In [17]:
print(sorted(list(pn_all_3.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [ ]:
save_dir = f"/Users/WangHao/Desktop/TODO/Data/figs"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for idx in range(len(pne_all)):
    assert (
        sorted(list(pne_all.keys()))[idx]
        == sorted(list(pne_all_1.keys()))[idx]
        == sorted(list(pne_all_2.keys()))[idx]
        == sorted(list(pne_all_3.keys()))[idx]
    )

    title = list(pne_all.keys())[idx][:-7]

    fig_x = list(pne_all[sorted(list(pne_all.keys()))[idx]].index)
    fig_y = list(pne_all[sorted(list(pne_all.keys()))[idx]].values)

    fig_x_1 = list(pne_all_1[sorted(list(pne_all_1.keys()))[idx]].index)
    fig_y_1 = list(pne_all_1[sorted(list(pne_all_1.keys()))[idx]].values)

    fig_x_2 = list(pne_all_2[sorted(list(pne_all_2.keys()))[idx]].index)
    fig_y_2 = list(pne_all_2[sorted(list(pne_all_2.keys()))[idx]].values)

    fig_x_3 = list(pne_all_3[sorted(list(pne_all_3.keys()))[idx]].index)
    fig_y_3 = list(pne_all_3[sorted(list(pne_all_3.keys()))[idx]].values)

    plot.rcParams.update({"font.size": 10})
    fig = plot.figure(idx, figsize=(18, 6), dpi=300)
    ax = fig.add_subplot(111)

    ax.set(title=title, xlabel="frame", ylabel="diameter stenosis", ylim=[0, 1])

    ax.plot(fig_x, fig_y, "m", label="label")

    ax.plot(fig_x_1, fig_y_1, "g", label="3dunet")

    ax.plot(fig_x_2, fig_y_2, "c", label="rvm")

    ax.plot(fig_x_3, fig_y_3, "b", label="transbts")

    ax.legend()
    ax.grid(True)

    fig.savefig(f"{save_dir}/{title}.png")

In [ ]:
save_dir = f"/Users/WangHao/Desktop/TODO/Data/figs"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for idx in range(len(pn_all)):
    assert (
        sorted(list(pn_all.keys()))[idx]
        == sorted(list(pn_all_1.keys()))[idx]
        == sorted(list(pn_all_2.keys()))[idx]
        == sorted(list(pn_all_3.keys()))[idx]
    )

    title = os.path.splitext(list(pn_all.keys())[idx])[0]

    fig_x = list(pn_all[sorted(list(pn_all.keys()))[idx]].index)
    fig_y = list(pn_all[sorted(list(pn_all.keys()))[idx]].values)

    fig_x_1 = list(pn_all_1[sorted(list(pn_all_1.keys()))[idx]].index)
    fig_y_1 = list(pn_all_1[sorted(list(pn_all_1.keys()))[idx]].values)

    fig_x_2 = list(pn_all_2[sorted(list(pn_all_2.keys()))[idx]].index)
    fig_y_2 = list(pn_all_2[sorted(list(pn_all_2.keys()))[idx]].values)

    fig_x_3 = list(pn_all_3[sorted(list(pn_all_3.keys()))[idx]].index)
    fig_y_3 = list(pn_all_3[sorted(list(pn_all_3.keys()))[idx]].values)

    fig = plot.figure(idx, figsize=(18, 4), dpi=300)
    plot.rcParams.update({"font.size": 8})
    ax = fig.add_subplot(141)
    ax.set(title="transbts", xlabel="frame", ylabel="diameter stenosis")
    ax.plot(fig_x, fig_y)
    ax.text(
        fig_y.index(max(fig_y)),
        max(fig_y),
        (fig_y.index(max(fig_y)), round(max(fig_y), 2)),
        color="r",
    )
    ax.grid(True)

    ax1 = fig.add_subplot(142)
    ax1.set(title="rvm", xlabel="frame", ylabel="diameter stenosis")
    ax1.plot(fig_x_1, fig_y_1)
    ax1.text(
        fig_y_1.index(max(fig_y_1)),
        max(fig_y_1),
        (fig_y_1.index(max(fig_y_1)), round(max(fig_y_1), 2)),
        color="r",
    )
    ax1.grid(True)

    ax2 = fig.add_subplot(143)
    ax2.set(title="3dunet", xlabel="frame", ylabel="diameter stenosis")
    ax2.plot(fig_x_2, fig_y_2)
    ax2.text(
        fig_y_2.index(max(fig_y_2)),
        max(fig_y_2),
        (fig_y_2.index(max(fig_y_2)), round(max(fig_y_2), 2)),
        color="r",
    )
    ax2.grid(True)

    ax3 = fig.add_subplot(144)
    ax3.set(title="label", xlabel="frame", ylabel="diameter stenosis")
    ax3.plot(fig_x_3, fig_y_3)
    ax3.text(
        fig_y_3.index(max(fig_y_3)),
        max(fig_y_3),
        (fig_y_3.index(max(fig_y_3)), round(max(fig_y_3), 2)),
        color="r",
    )

    ax3.grid(True)

    fig.savefig(f"{save_dir}/{title}.png")

In [34]:
a = np.array([])
print(list(a) == [])
print(len(a))
print(np.size(a))
b = np.array(1)
print(np.isscalar(b))
print(len(b.shape))
print(np.size(b))

True
0
0
False
0
1
